# Set Up

Here is the standard bit of code that will set up the Intelligent Plant App Store and Data Core API clients for a notebook

In [ ]:
import intelligent_plant.app_store_client as app_store_client
import os
app_store = app_store_client.AppStoreClient(os.environ["ACCESS_TOKEN"])
data_core = app_store.get_data_core_client()

When you log in your access token is put in the enviroment variable `ACCESS_TOKEN`. This code initialises the app store client with your access token.

**All client methods throw an HTTPError if there is an error making the request**

This Jupyter Lab is re-configured with a variety of libraries for data science. However, if there is another Python package you would like to use you can install it using:

In [ ]:
%pip install tensorflow

You may need to restart the kernal in order for the changes to take effect.

# App Store

## Get Data Core Client

The app store client contains a function `get_data_core_client(..)` that instantiates a data core client with the same access information as that app store client. This is used to get the data core client needed to make data queries.

In [ ]:
app_store.get_data_core_client()

## Get User Info

The `get_user_info()` function which can be used to get the information of the currently logged in user.

In [ ]:
app_store.get_user_info()

## Get User Balance

The `get_user_balance()` function gets the balance of the currently authenticated user.

In [ ]:
app_store.get_user_balance()

## Debit Account

The `debit_account(..)` function debits the currently autheticated user's app store account.

Parameters:
- `amount` The amount of credits to debit from the account

Returns:
The transaction reference for the debit transaction

In [ ]:
ref = app_store.debit_account(0) #this ref will be used in the refund example
ref

# Refund Account
The `refund_account(..)` function refunds a previous transaction.

Parameters:
- `transaction_ref` The reference of the transaction to refund

Returns the HTTP response object of the request.

In [ ]:
app_store.refund_account(ref)

# Data Core

To get the data core client object use the `app_store.get_data_core_client()` function.

In [ ]:
data_core = app_store.get_data_core_client()

**These examples use IP Datasource 2 as the data source please make sure that it is authorized for the Jupyter app**

https://appstore.intelligentplant.com/Security/Apps

## Get Data Sources

The get data sources method can be used to query the data sources available to the currently logged in user.
Parameters:
- `filters` A list of filters that should be applied to the search (if unsure try `["*"]` to get all data sources)

Returns:
A list of of available data sources and their meta data.

**When making other queries you want to use the fully qualified data source name**

`datasource["Name"]["QualifiedName"]`

In [ ]:
data_core.get_data_sources(["*"])

## Get Tags

The `get_tags(..)` function is used to query a data source for the available tags.

A tag is the name and meta data of a single data stream.

Tag searches are paginated to prevent the request sizes from getting too large.

Parameters:
- `dsn` The fully qualified name of the data source you want to query, see `get_data_sources(..)` for more information
- `page` (Optional, default = 1) The number of the results page you would like to get
- `page_size` (Optional, default = 20) The number of results to return per page
- `filters` (Optional, default = {}) A dictionary of filters to apply where the key is the field name (e.g. name, description, unit) and the value is the filter to apply (which can include the wildcards `?` and `*`)

Wildcards:
- `?` Any single character
- `*` Ay number (including 0) of any character

Returns:
A list of available tag objects

Data source information is the in 'DataSourceName' dictionary.

The tag information is in the 'Tags' list.

**In data queryies you want to use the values of the tag ID field**

In [ ]:
data_core.get_tags("IP Datasource 2")

## Getting Data

There are multiple different functions that are used to query datqa sources depending on the type of data you want to get and how it should be processed.

### Tag Map

All of the data query functions take a `tags` parameter. This parameter is used to specify which tags you want to query (and the data source they exist on).

The tag map should be a dictionary where the data source name is the key and the value is a list of tags on that data source.

For example to query the "hed and "flow" tags on IP Data source and the "Air Scour Blowers_Outlet_PI" and "Antifoam Inj Pump B_PI" tags on IP Data source 2 the `tags` parameter would be:

In [ ]:
tags = {
    "IP Datasource": [
        "head",
        "flow"
    ],
    "IP Datasource 2": [
        "Air Scour Blowers_Outlet_PI",
        "Antifoam Inj Pump B_PI"
    ]
}

### Time Stamps

Data core allows you to use absolute or relative time stamps in your data queries.

When making a query the time parameters should be a string in one of these formats.

#### Absolute Time Stamps

Absoulte time stamps are in the ISO 8601 format:

In [ ]:
from datetime import datetime
datetime.now().isoformat()

#### Relative Time Stamps

Relative timestamps can be used to specify a time from the current time.

The `*` character indicates now and the `-` is used to subtract time from it.

Some examples:
- `"*"` is now 
- `"*-12h"` is 12 hours ago
- `"*-10d"` is 10 days ago
- `"*-1y"` is 1 year ago

### Responses

#### Tag Value

All of the get data function return values using the tag value objects that looks like this:

`{
    "TagName": "<tag name>",
    "UtcSampleTime": "<yyyy-MM-ddTHH:mm:ss.fffZ>",
    "NumericValue": "<some number>",
    "TextValue": "<some text>",
    "IsNumeric": true | false,
    "Status": "Good" | "Bad" | "Uncertain",
    "Unit": "<some unit>",
    "Notes": "<optional notes>",
    "Error": "<optional error message>",
    "HasError": true | false,
    "Properties": {
        "prop1": <some value>,
        ...
        "propN": <some value>
    }
}`

#### Historical Tag Value

Historical data requests return data in a historical tag values object:

`{
    "TagName": "<tag name>",
    // Display hint when visualizing the data i.e straight line between points or trailing/leading edge
    "DisplayType": "Unspecified" | "Interpolate" | "TrailingEdge" | "LeadingEdge",
    "Values": [
        <DataCoreTagValue>,
        ...
        <DataCoreTagValue>
    ]
}`

#### Response Dictionary

All responses organise the tag values in a dictionary where the key is the data source name and the value is a nested dictionary. The nested dictionary has the tag name as the key and the tag value object (for snapshot) or historical tag value object as the value.

`{
    "data source name 1": {
        "tag_name_1_1": <DataCoreTagValue or DataCoreHistoricalTagValue>,
        ...
        "tag_name_1_N": <DataCoreTagValue or DataCoreHistoricalTagValue>
    },
    "data source name N": {
        "tag_name_N_1": <DataCoreTagValue or DataCoreHistoricalTagValue>,
        ...
        "tag_name_N_N": <DataCoreTagValue or DataCoreHistoricalTagValue>
    }
}`

### Get Plot Data

Plot data should be used when you want to draw a plot.

The way plot data is processed a special type of interploation that ensures that the maximum and minium values are preserved so that you don't lose an peaks.

To query plot data use the `get_plot_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `intervals` The number of intervals the time range should be divided into

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [ ]:
data_core.get_plot_data({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", 10)

### Get Processed Data

The get process data function is used to fecth data with a certain function applied. This allows the samples to have a consistent interval.

To query processed data use the `get_processed_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `sample_interval` A string representing the length of time each sample should represent, this uses a similar format to the realtive time stamps e.g. `"1d"`, `"1h"` etc.
- `data_function` The data processing function to use for the query. INTERP/AVG/MIN/MAX

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [ ]:
data_core.get_processed_data({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", "1d", "interp")

### Get Raw Data

The get raw data function is used to get the raw values of the data in a certain time range, without any kind of data processing. Warning: this can causes extremely large amaounts of data t be returned.

To query raw data use the `get_raw_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `point_count` the maximum number of data points to return. 0 will fetch as many as possible. 

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [ ]:
data_core.get_raw_data({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", 10)

### Get Data At Times

The get data at times function allows you to qery data and specify the exact sample times you wuld like to have returned.

To query for specific times use the `get_data_at_times(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `utc_sample_times` A list of UTC sample point to get the data for (in the ISO format)

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [ ]:
data_core.get_data_at_times({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]}, [datetime.utcnow().isoformat()])

### Get Snapshot Data

The get the current value of some tags use the `get_snapshot_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a tag value object as the value.

In [ ]:
data_core.get_snapshot_data({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]})

## Data Frame Utility

The intelligent plant package includes a utility function that converts the response object into a pandas data frame.

**This function assumes that the timestamps of all the returned sample for each tag line up which isn't true for all queries and data functions**

Parmaters:
- `result` The result object from a data core request

Returns a data frame of the same data with tag names as column names

Data frames can be used by a variety of data analysis and machine learning frameworks , see the other demo notebooks for more information

In [ ]:
import intelligent_plant.utility as utility

In [ ]:
resp = data_core.get_processed_data({"IP Datasource 2": ["Air Scour Blowers_Outlet_PI"]}, "*-30d", "*", "1d", "interp")
df = utility.query_result_to_data_frame(resp)
df